In [2]:
import sys
i = 0
states = {}
with open('mate.db', 'r') as f:
    pos = None
    while l := f.readline():
        if len(l) > 0 and l[0] == '#':
            continue
        cs = l.split()
        #print('cs', cs)
        if len(cs) > 0 and cs[0] == 'sfen':
            if len(cs) != 5 or cs[4] != '0' or cs[2] not in 'bw':
                print(l)
                continue
            pos = (cs[1], cs[2], cs[3])
            if pos in states:
                print('dup', pos)
                continue
            states[pos] = []
        else:
            if len(cs) != 3:
                print('invalid', cs)
                continue
            if pos[1] == 'w':
                if int(cs[2]) >= -30000:
                    print('invalid', cs)
                    continue
            else:
                if pos[1] == 'b':
                    if int(cs[2]) <= 30000:
                        print('invalid', cs)
                        continue
            states[pos].append(cs[0])
        i += 1
print('i', i)

i 95844910


In [3]:
print('len(states)=', len(states))
pos2i = {}
i2pos = []
for pos in states:
    pos2i[pos] = len(i2pos)
    i2pos.append(pos)    

len(states)= 38339458


初期配置は，
```
sfen p+nks+l/5/5/5/+LSK+N+P b - 0
```
だと思っていたが，これは含まれずに，
```
sfen p+nks+l/5/5/4+P/+LSK+N1 w - 0
```
のようだった．


In [4]:
initpos = ('p+nks+l/5/5/4+P/+LSK+N1','w','-')

In [ ]:
states[initpos]

In [5]:
def str2board(s):
    ans = [[''] * 5 for _ in range(5)]
    for i, l in enumerate(s.split('/')):
        x = 0
        j = 0
        while x < 5:
            if l[j].isdigit():
                for k in range(int(l[j])):
                    ans[i][x + k] = ''
                x += int(l[j])
                j += 1
            else:
                if l[j] == '+':
                    ans[i][x] = l[j:(j + 2)]
                    j += 2
                else:
                    ans[i][x] = l[j]
                    j += 1
                x += 1
    return ans

def board2str(b):
    ans = []
    for y in range(5):
        l = []
        c = 0
        for x in range(5):
            if b[y][x] == '':
                c += 1
            else:
                if c > 0:
                    l.append(str(c))
                    c = 0
                l.append(b[y][x])
        if c > 0:
            l.append(str(c))
        ans.append(''.join(l))
    return '/'.join(ans)

pstr = 'snlpk'
pstrs = [''.join(c.upper() for c in pstr), pstr]

def str2hands(s):
    ans = [[0] * 5 for _ in range(2)]
    for i in range(2):
        for j, c in enumerate(pstr):
            if i == 0:
                c = c.upper()
            if c in s:
                k = s.index(c)
                if k > 0 and s[k - 1] == '2':
                    ans[i][j] = 2
                else:
                    ans[i][j] = 1
    return ans

def hands2str(hand):
    ans = []
    for i in range(2):
        for j in range(5):
            if hand[i][j] > 0:
                if hand[i][j] == 2:
                    ans.append('2')
                ans.append(pstrs[i][j])
    if len(ans) == 0:
        return '-'
    return ''.join(ans)


def str2turn(s):
    return 0 if s == 'b' else 1

def turn2str(turn):
    return 'bw'[turn]


def p2turn(p):
    if p == '':
        return ''
    elif p[-1].isupper():
        return 0
    return 1


def captured(p):
    p = p[-1]
    if p.isupper():
        return p.lower()
    return p.upper()


p2moves = {0 : {
    'L' : [(0, -1, 1)],
    '+L' : [(-1, -1, 0), (0, -1, 0), (1, -1, 0),(-1, 0, 0), (1, 0, 0),(0, 1, 0)],
    'S' : [(-1, -1, 0), (0, -1, 0), (1, -1, 0),(-1, 1, 0),(1, 1, 0)],
    '+S' : [(1, 1, 1), (1, -1, 1), (-1, 1, 1), (-1, -1, 1)],
    'K' : [(-1, -1, 0), (0, -1, 0), (1, -1, 0),(-1, 0, 0), (1, 0, 0), (-1, 1, 0),(0, 1, 0), (1, 1, 0)],
    'N' : [(-1, -2, 0), (1, -2, 0)],
    '+N' :  [(-1, -1, 0), (0, -1, 0), (1, -1, 0),(-1, 0, 0), (1, 0, 0),(0, 1, 0)],
    'P': [(0, -1, 0)],
    '+P' : [(0, -1, 1), (0, 1, 1), (1, 0, 1), (-1, 0, 1)]}}
p2moves[1] = {}
for p in p2moves[0]:
    wp = p.lower() if len(p) == 1 else '+' + p[1].lower()
    ms = [(x, -y, islong) for x, y, islong in p2moves[0][p]]
    p2moves[1][wp] = ms

ystr = 'abcde'
def c2x(c):
    return 5 - int(c)


def c2y(c):
    return ord(c) - ord('a')


def x2c(x):
    return str(5 - x)

def y2c(y):
    return ystr[y]


def makemove(x, y, x1, y1, p):
    pstr = '' if p.upper() == 'K' else ('-' if p[0] == '+' else '+')
    return f'{x2c(x)}{y2c(y)}{x2c(x1)}{y2c(y1)}{pstr}'

def makedrops(p, x, y):
    p = p[:-1] + p[-1].upper()
    m = f'{p}*{x2c(x)}{y2c(y)}'
    return [m, '+' + m]


def genmoves(pos):
    b, turn, hands = str2board(pos[0]), str2turn(pos[1]), str2hands(pos[2])
    #print('b', b, 'turn', turn, 'hands', hands)
    ans = []
    for y in range(5):
        for x in range(5):
            if b[y][x] == '':
                for i in range(5):
                    if hands[turn][i] > 0:
                        ans.extend(makedrops(pstrs[turn][i], x, y))
            elif p2turn(b[y][x]) == turn:
                #print('x,y,b[y][x],p2moves', (x, y, b[y][x]),p2moves[turn][b[y][x]])
                for dx, dy, islong in p2moves[turn][b[y][x]]:
                    x1, y1 = x + dx, y + dy
                    if islong:
                        while 0 <= x1 < 5 and 0 <= y1 < 5 and p2turn(b[y1][x1]) == '':
                            ans.append(makemove(x, y, x1, y1, b[y][x]))
                            x1 += dx
                            y1 += dy
                    if 0 <= x1 < 5 and 0 <= y1 < 5 and p2turn(b[y1][x1]) != turn:
                        ans.append(makemove(x, y, x1, y1, b[y][x]))
    return ans

def remove_from_hands(hands, p):
    op = p[1] if p[0] == '+' else p[0]
    newhands = [l[:] for l in hands]
    owner = 0 if op.isupper() else 1
    ptype = pstr.index(op.lower())
    assert hands[owner][ptype] > 0
    newhands[owner][ptype] -= 1
    return newhands


def add_to_hands(hands, p):
    newhands = [l[:] for l in hands]
    owner = 0 if p.isupper() else 1
    ptype = pstr.index(p.lower())
    assert hands[owner][ptype] < 2
    newhands[owner][ptype] += 1
    return newhands


def applymove(pos, move):
    b, turn, hands = str2board(pos[0]), str2turn(pos[1]), str2hands(pos[2])
    if '*' in move:
        x, y = c2x(move[-2]), c2y(move[-1])
        p = move[:2] if move[0] == '+' else move[0]
        b[y][x] = p if turn == 0 else p.lower()
        #print('pos, move', (pos, move, (p[-1] if turn == 0 else p[-1].lower())))
        hands = remove_from_hands(hands, p[-1] if turn == 0 else p[-1].lower())
    else:
        sx, sy = c2x(move[0]), c2y(move[1])
        dx, dy = c2x(move[2]), c2y(move[3])
        dp = b[dy][dx]
        if dp != '':
            hands = add_to_hands(hands, captured(dp))
        p = b[sy][sx]
        b[sy][sx] = ''
        afterp = '+' + p if len(p) == 1 and p not in 'kK' else p[-1]
        b[dy][dx] = afterp
    turn = 1 - turn
    return (board2str(b), turn2str(turn), hands2str(hands))


def is_check(b, turn):
    for y in range(5):
        for x in range(5):
            if p2turn(b[y][x]) == turn:
                #print('x,y,b[y][x],p2moves', (x, y, b[y][x]),p2moves[turn][b[y][x]])
                for dx, dy, islong in p2moves[turn][b[y][x]]:
                    x1, y1 = x + dx, y + dy
                    if islong:
                        while 0 <= x1 < 5 and 0 <= y1 < 5 and p2turn(b[y1][x1]) == '':
                            x1 += dx
                            y1 += dy
                    if 0 <= x1 < 5 and 0 <= y1 < 5 and p2turn(b[y1][x1]) != turn and b[y1][x1].lower() == 'k':
                        return True
    return False


def is_suicide_move(pos, move):
    newpos = applymove(pos, move)
    #print('newpos', newpos)
    turn = str2turn(newpos[1])
    b = str2board(newpos[0])
    return is_check(b, turn)


def genmoves_legal(pos):
    return [m for m in genmoves(pos) if not is_suicide_move(pos, m)]


In [6]:
n = len(states)
g = [[] for _ in range(n)]
rg = [[] for _ in range(n)]
for i, pos in enumerate(i2pos):
    for m in states[pos]:
        pos1 = applymove(pos, m)
        if pos1 in pos2i:
            j = pos2i[pos1]
            g[i].append(j)
            rg[j].append(i)
            



In [7]:
visited = [False] * n
def traverse(i):
    if visited[i]:
        return
    visited[i] = True
    for j in g[i]:
        traverse(j)
traverse(pos2i[initpos])
print(sum(visited), len(visited))


34953776 38339458


In [10]:
cb, cw = 0, 0
for i, pos in enumerate(i2pos):
    if not visited[i]:
        if pos[1] == 'w':
            cw += 1
        else:
            cb += 1
print('cw, cb', (cw, cb))

cw, cb (2446887, 938795)


In [12]:
c = 0
for i, pos in enumerate(i2pos):
    if visited[i]:
        moves = genmoves_legal(pos)
        for m in moves:
            if m not in states[pos]:
                print ('pos', pos, 'm', m, 'moves', (states[pos], moves))
                c += 1
    if c > 100:
        break

            

pos ('2k2/S3+P/2n2/2K2/5', 'b', 'SN2LP') m S*5a moves (['+P*4a'], ['S*5a', '+S*5a', 'N*5a', '+N*5a', 'L*5a', '+L*5a', 'P*5a', '+P*5a', 'S*4a', '+S*4a', 'N*4a', '+N*4a', 'L*4a', '+L*4a', 'P*4a', '+P*4a', 'S*2a', '+S*2a', 'N*2a', '+N*2a', 'L*2a', '+L*2a', 'P*2a', '+P*2a', 'S*1a', '+S*1a', 'N*1a', '+N*1a', 'L*1a', '+L*1a', 'P*1a', '+P*1a', '5b5a+', '5b4a+', '5b4c+', 'S*4b', '+S*4b', 'N*4b', '+N*4b', 'L*4b', '+L*4b', 'P*4b', '+P*4b', 'S*3b', '+S*3b', 'N*3b', '+N*3b', 'L*3b', '+L*3b', 'P*3b', '+P*3b', 'S*2b', '+S*2b', 'N*2b', '+N*2b', 'L*2b', '+L*2b', 'P*2b', '+P*2b', '1b1a-', '1b1c-', '1b1d-', '1b1e-', '1b2b-', '1b3b-', '1b4b-', 'S*5c', '+S*5c', 'N*5c', '+N*5c', 'L*5c', '+L*5c', 'P*5c', '+P*5c', 'S*4c', '+S*4c', 'N*4c', '+N*4c', 'L*4c', '+L*4c', 'P*4c', '+P*4c', 'S*2c', '+S*2c', 'N*2c', '+N*2c', 'L*2c', '+L*2c', 'P*2c', '+P*2c', 'S*1c', '+S*1c', 'N*1c', '+N*1c', 'L*1c', '+L*1c', 'P*1c', '+P*1c', 'S*5d', '+S*5d', 'N*5d', '+N*5d', 'L*5d', '+L*5d', 'P*5d', '+P*5d', 'S*4d', '+S*4d', 'N*4d', '+

In [209]:
# moveした後のpositionが存在することの確認
i = 0
j = 0
for pos in states:
    j += 1
    if j % 10000 == 0:
        print('j', j)
    if pos[1] == 'w':
        continue
    for m in states[pos]:
        pos1 = applymove(pos, m)
        if pos1 not in states:
            if pos1[1] == 'b' and has_mate(pos1, 3):
                #print('mate', 'pos', pos, 'm', m, 'pos1', pos1)
                pass
            elif pos1[1] == 'w' and noescape(pos1, 4):
                pass
            else:
                print('nomate', 'pos', pos, 'm', m, 'pos1', pos1)
                i += 1
    if i > 1000:
        break

j 10000
j 20000
j 30000
j 40000
j 50000
j 60000
j 70000
j 80000
j 90000
j 100000
j 110000
j 120000
j 130000
j 140000
j 150000
j 160000
j 170000
j 180000
j 190000
j 200000
j 210000
j 220000
j 230000
j 240000
j 250000
j 260000
j 270000
j 280000
j 290000
j 300000
j 310000
j 320000
j 330000
j 340000
j 350000
j 360000
j 370000
j 380000
j 390000
j 400000
j 410000
j 420000
j 430000
j 440000
j 450000
j 460000
j 470000
j 480000
j 490000
j 500000
j 510000
j 520000
j 530000
j 540000
j 550000
j 560000
j 570000
j 580000
j 590000
j 600000
j 610000
j 620000
j 630000
j 640000
j 650000
j 660000
j 670000
j 680000
j 690000
j 700000
j 710000
j 720000
j 730000
j 740000
j 750000
j 760000
j 770000
j 780000
j 790000
j 800000
j 810000
j 820000
j 830000
j 840000
j 850000
j 860000
j 870000
j 880000
j 890000
j 900000
j 910000
j 920000
j 930000
j 940000
j 950000
j 960000
j 970000
j 980000
j 990000
j 1000000
j 1010000
j 1020000
j 1030000
j 1040000
j 1050000
j 1060000
j 1070000
j 1080000
j 1090000
j 1100000
j 111000

In [186]:
len(states)

38339458

is_suicide_move をテストする．i =

In [ ]:
# ('5/Sk2+P/2n2/2K2/5', 'w', 'SN2LP')  '4b5b' '3c4e+' はsuicide
pos = ('5/Sk2+P/2n2/2K2/5', 'w', 'SN2LP')
print(applymove(pos, '4b5b'))
print(is_suicide_move(pos, '4b5b'))
print(is_suicide_move(pos, '3c4e+'))



一手詰みがあるかどうかを判定する


In [189]:
def has_mate1(pos):
    moves = genmoves(pos)
    for m in moves:
        pos1 = applymove(pos, m)
        if not is_check(str2board(pos1[0]), str2turn(pos[1])):
            continue
        moves2 = genmoves_legal(pos1)
        if len(moves2) == 0:
            return True
    return False


def noescape(pos, depth):
    moves = genmoves_legal(pos)
    for m in moves:
        pos1 = applymove(pos, m)
        if not has_mate(pos1, depth - 1):
            return False
    return True


def has_mate(pos, depth):
    if depth <= 1:
        return has_mate1(pos)
    moves = genmoves(pos)
    for m in moves:
        pos1 = applymove(pos, m)
        if not is_check(str2board(pos1[0]), str2turn(pos[1])):
            continue
        if noescape(pos, depth - 1):
            return True
    return False
       

検証したいこと
- move generator, applymoveのバグ
- ステイルメイトの扱い
- 千日手チェックをちゃんとおこなっているか?


# initposからmove gen して，applymove してみる．
for m in genmoves(initpos):
    print('m', m, applymove(initpos, m))


In [ ]:
genmoves_legal(initpos)

In [ ]:
# initposからmove gen して，applymove してみる．
for m in genmoves_legal(initpos):
    print('m', m, applymove(initpos, m))


In [164]:
# s ('4P/+p2k1/5/1L+S2/2K+N1', 'w', 'Snl') から，genmove して，Sを打とうとする．
pos = ('4P/+p2k1/5/1L+S2/2K+N1', 'w', 'Snl')
print(genmoves(pos))


['N*5a', '+N*5a', 'L*5a', '+L*5a', 'N*4a', '+N*4a', 'L*4a', '+L*4a', 'N*3a', '+N*3a', 'L*3a', '+L*3a', 'N*2a', '+N*2a', 'L*2a', '+L*2a', '5b5c-', '5b5d-', '5b5e-', '5b5a-', '5b4b-', '5b3b-', 'N*4b', '+N*4b', 'L*4b', '+L*4b', 'N*3b', '+N*3b', 'L*3b', '+L*3b', '2b3c', '2b2c', '2b1c', '2b3b', '2b1b', '2b3a', '2b2a', '2b1a', 'N*1b', '+N*1b', 'L*1b', '+L*1b', 'N*5c', '+N*5c', 'L*5c', '+L*5c', 'N*4c', '+N*4c', 'L*4c', '+L*4c', 'N*3c', '+N*3c', 'L*3c', '+L*3c', 'N*2c', '+N*2c', 'L*2c', '+L*2c', 'N*1c', '+N*1c', 'L*1c', '+L*1c', 'N*5d', '+N*5d', 'L*5d', '+L*5d', 'N*2d', '+N*2d', 'L*2d', '+L*2d', 'N*1d', '+N*1d', 'L*1d', '+L*1d', 'N*5e', '+N*5e', 'L*5e', '+L*5e', 'N*4e', '+N*4e', 'L*4e', '+L*4e', 'N*1e', '+N*1e', 'L*1e', '+L*1e']


s ('5/4k/3+N1/1LS2/2K1+s', 'w', 'LPnp') moves ([], ['1b2a', '1b1a'])
これは，applymove直後は合法のような気がするが



In [165]:
pos = ('5/4k/3+N1/1LS2/2K1+s', 'w', 'LPnp')
newpos = applymove(pos, '1b2a')
print(newpos)
print(str2board(newpos[0]))

('3k1/5/3+N1/1LS2/2K1+s', 'b', 'LPnp')
[['', '', '', 'k', ''], ['', '', '', '', ''], ['', '', '', '+N', ''], ['', 'L', 'S', '', ''], ['', '', 'K', '', '+s']]


sfenにすると? '5/4k/3+N1/1LS2/2K1+s w LPnp'

In [198]:
i = 0
j = 0
for s in states:
    j += 1
    if j % 100 == 0:
        print(j)
    if s[1] == 'w':
        #if set(states[s]) != set(genmoves(s)):
        #    print('s', s, 'moves', (states[s], genmoves(s)))
        #    i += 1
        moves = genmoves_legal(s)
        if set(states[s]) != set(moves):
            for m in moves:
                if m not in states[s]:
                    pos1 = applymove(s, m)
                    mate = False
                    for d in range(1, 20, 2):
                        if has_mate(pos1, d):
                            print('mate: d', d, 'pos1', pos1)
                            mate = True
                            break
                        else:
                            print('nomate: d', d, 'pos1', pos1)
                    if mate:
                        continue
                    print('s', s, 'm', m, 'moves', (states[s], moves))
                    i += 1
            for m in states[s]:
                if m not in moves:
                    print('s', s, 'm', m, 'moves', (states[s], moves))
                    i += 1
        #print('moves', moves, 'states[s]', states[s])
        #for m in states[s]:
        #    if m not in moves:
        #        print('s', s, '[m]', [m], 'moves', moves)
        #       i += 1
        #print('end check')
    else:
        #if j < 10764870:
        #    continue
        #print(j, s)

        if len(states[s]) != 1 or states[s][0] not in genmoves_legal(s):
            print('s', s, 'moves', (states[s], genmoves_legal(s)))
            i += 1
    if i > 100:
        
        break



mate: d 1 pos1 ('3k1/5/3+N1/1LS2/2K1+s', 'b', 'LPnp')
mate: d 1 pos1 ('4k/5/3+N1/1LS2/2K1+s', 'b', 'LPnp')
nomate: d 1 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 3 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 5 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 7 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 9 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 11 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 13 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 15 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 17 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
nomate: d 19 pos1 ('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
s ('k4/3L+N/3K1/p3+P/4+S', 'w', 'SNL') m 5a5b moves ([], ['5a5b', '5a4a'])
mate: d 1 pos1 ('1k3/3L+N/3K1/p3+P/4+S', 'b', 'SNL')
mate: d 1 pos1 ('1+S2l/3k1/4s/1LKN1/4+P', 'b', 'Np')
mate: d 1 pos1 ('1+S1kl/5/4s/1LKN1/4+P', 'b', 'Np')
nomate: d 1 pos1 ('4P/1k2+n/4n/2+S2/2K2', 'b', 'LPsl')
nomate: d 3 pos1 ('4P/1k2+n/4n/2+S2/2K2', 'b', 'LPsl')
nomat

KeyboardInterrupt: 

In [ ]:
 ('5/4k/3+N1/1LS2/2K1+s', 'w', 'LPnp')

In [170]:
s = ('1+L2k/3+N+p/3+N1/5/pSK2', 'w', 'SL')
print(genmoves(s))
pos1 = applymove(s, '1b2a')
print(has_mate1(pos1))
print(has_mate3(pos1))

['1a2b', '1a2a', '1b1c-', '1b1d-', '1b1e-', '1b2b-']
True
True


In [ ]:
pos = ('1k2P/1+L2+n/4n/2+S2/2K2', 'w', 'LPs')
pos1 = applymove(pos, '4a4b')

move generator のデバッグ
s ('3k1/n2l1/1K3/1+s+S1+P/5', 'w', 'NLP') [m] ['4d3c-'] moves ['2a3b', '2a1b', '2a3a', '2a1a', '2b2c+', '2b2d+', '2b2e+', '4d5e-', '4d3e-', '4d5c-', '4d5e-']
4d の +s が?3cにmoveしない．
角の動き



In [176]:
pos = ('k4/3L+N/3K1/p3+P/4+S', 'w', 'SNL')
pos1 = applymove(pos, '5a5b')
print(pos1)
print(has_mate1(pos1))
print(has_mate3(pos1))


('5/k2L+N/3K1/p3+P/4+S', 'b', 'SNL')
False
False


In [ ]:
'+l'.upper()

In [ ]:
i = 0
for s in states:
    if len(s[2]) > 5:
        print(s)
        i += 1
        if i > 50:
            break

In [177]:
len(states)

38339458

In [178]:
54728872-18443888

36284984

In [180]:
c = 0
for s in states:
    if len(states[s]) == 0:
        c += 1
print(c)        

2054473
